# Conway's Reverse Game of Life

# Keras - 1 Attempt

# Attempt 1

In [ ]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('conway-s-reverse-game-of-life/sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
from tensorflow.keras import layers, models
model = models.Sequential()

if n_samples > 10000 and n_features > 100:
    layer_sizes = [min(int(n_features * i), 1024) for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [sz for sz in layer_sizes if sz >= 16]
    for sz in layer_sizes:
        model.add(layers.Dense(sz, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))
else:
    for sz in [min(n_features * 2, 128), min(n_features, 64)]:
        model.add(layers.Dense(sz, activation='relu'))
        model.add(layers.Dropout(0.3))

# Output layer
model.add(layers.Dense(output_units, activation='sigmoid'))

# Compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Epoch 1/100


313/313 - 7s - 22ms/step - accuracy: 0.0024 - loss: 0.5049 - precision_2: 0.2181 - recall_2: 0.2383 - val_accuracy: 0.0043 - val_loss: 0.3754 - val_precision_2: 0.3676 - val_recall_2: 0.0031
Epoch 2/100


313/313 - 3s - 11ms/step - accuracy: 0.0088 - loss: 0.3707 - precision_2: 0.3324 - recall_2: 0.0108 - val_accuracy: 0.0185 - val_loss: 0.3523 - val_precision_2: 0.5043 - val_recall_2: 0.0016
Epoch 3/100


313/313 - 3s - 11ms/step - accuracy: 0.0174 - loss: 0.3572 - precision_2: 0.4158 - recall_2: 0.0101 - val_accuracy: 0.0205 - val_loss: 0.3440 - val_precision_2: 0.5310 - val_recall_2: 0.0061
Epoch 4/100


313/313 - 4s - 11ms/step - accuracy: 0.0215 - loss: 0.3509 - precision_2: 0.4400 - recall_2: 0.0157 - val_accuracy: 0.0217 - val_loss: 0.3394 - val_precision_2: 0.5274 - val_recall_2: 0.0096
Epoch 5/100


...

313/313 - 5s - 15ms/step - accuracy: 0.0439 - loss: 0.3163 - precision_2: 0.5156 - recall_2: 0.0619 - val_accuracy: 0.0478 - val_loss: 0.3213 - val_precision_2: 0.6792 - val_recall_2: 0.0337
Epoch 79/100
313/313 - 4s - 14ms/step - accuracy: 0.0448 - loss: 0.3163 - precision_2: 0.5164 - recall_2: 0.0622 - val_accuracy: 0.0441 - val_loss: 0.3214 - val_precision_2: 0.6836 - val_recall_2: 0.0338
Epoch 80/100


313/313 - 4s - 14ms/step - accuracy: 0.0451 - loss: 0.3163 - precision_2: 0.5174 - recall_2: 0.0621 - val_accuracy: 0.0439 - val_loss: 0.3213 - val_precision_2: 0.6854 - val_recall_2: 0.0336
Epoch 81/100
313/313 - 4s - 14ms/step - accuracy: 0.0439 - loss: 0.3161 - precision_2: 0.5174 - recall_2: 0.0618 - val_accuracy: 0.0467 - val_loss: 0.3213 - val_precision_2: 0.6847 - val_recall_2: 0.0326
Epoch 82/100


313/313 - 5s - 15ms/step - accuracy: 0.0460 - loss: 0.3161 - precision_2: 0.5170 - recall_2: 0.0627 - val_accuracy: 0.0476 - val_loss: 0.3213 - val_precision_2: 0.6837 - val_recall_2: 0.0324
Epoch 83/100


313/313 - 5s - 14ms/step - accuracy: 0.0440 - loss: 0.3160 - precision_2: 0.5168 - recall_2: 0.0628 - val_accuracy: 0.0460 - val_loss: 0.3213 - val_precision_2: 0.6801 - val_recall_2: 0.0342
Epoch 84/100


313/313 - 4s - 14ms/step - accuracy: 0.0448 - loss: 0.3157 - precision_2: 0.5171 - recall_2: 0.0636 - val_accuracy: 0.0443 - val_loss: 0.3212 - val_precision_2: 0.6783 - val_recall_2: 0.0342
Epoch 85/100
313/313 - 4s - 14ms/step - accuracy: 0.0466 - loss: 0.3158 - precision_2: 0.5179 - recall_2: 0.0634 - val_accuracy: 0.0455 - val_loss: 0.3213 - val_precision_2: 0.6848 - val_recall_2: 0.0332
Epoch 86/100
313/313 - 5s - 15ms/step - accuracy: 0.0456 - loss: 0.3156 - precision_2: 0.5160 - recall_2: 0.0635 - val_accuracy: 0.0435 - val_loss: 0.3213 - val_precision_2: 0.6754 - val_recall_2: 0.0358
Epoch 87/100
313/313 - 4s - 14ms/step - accuracy: 0.0449 - loss: 0.3155 - precision_2: 0.5165 - recall_2: 0.0635 - val_accuracy: 0.0420 - val_loss: 0.3213 - val_precision_2: 0.6819 - val_recall_2: 0.0339
Epoch 88/100
313/313 - 4s - 14ms/step - accuracy: 0.0466 - loss: 0.3155 - precision_2: 0.5172 - recall_2: 0.0640 - val_accuracy: 0.0485 - val_loss: 0.3213 - val_precision_2: 0.6829 - val_recall_2: 

# Conway's Reverse Game of Life

## Keras Tuner - 2 Attempts

## Attempt 1 

In [ ]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('conway-s-reverse-game-of-life/train.csv.zip')
test_df = pd.read_csv('conway-s-reverse-game-of-life/test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('conway-s-reverse-game-of-life/sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = X_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        output_units = len(target_columns)
        x = Dense(output_units, activation='sigmoid')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
        return model

# Tuner
bs = 64  # Example batch size
ep = 100  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=1,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=1,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

NameError: name 'X_val_proc' is not defined

## Attempt 2

In [1]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Split the data for validation
X_train_proc, X_val_proc, y_train, y_val = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=seed)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = X_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        output_units = len(target_columns)
        x = Dense(output_units, activation='sigmoid')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
        return model

# Tuner
bs = 64  # Example batch size
ep = 100  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
start_time = time.time()  # Start time for training
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
end_time = time.time()  # End time for training
print(f"Training time: {end_time - start_time} seconds")

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 06m 24s]
val_loss: 0.3264213502407074

Best val_loss So Far: 0.31926238536834717
Total elapsed time: 00h 47m 08s
Epoch 1/100


625/625 - 8s - 13ms/step - Precision: 0.3752 - Recall: 0.0316 - accuracy: 0.0237 - loss: 0.3584 - val_Precision: 0.5468 - val_Recall: 0.0499 - val_accuracy: 0.0399 - val_loss: 0.3298
Epoch 2/100


625/625 - 3s - 5ms/step - Precision: 0.5175 - Recall: 0.0648 - accuracy: 0.0402 - loss: 0.3331 - val_Precision: 0.5659 - val_Recall: 0.0670 - val_accuracy: 0.0396 - val_loss: 0.3242
Epoch 3/100


625/625 - 3s - 5ms/step - Precision: 0.5364 - Recall: 0.0739 - accuracy: 0.0451 - loss: 0.3279 - val_Precision: 0.5782 - val_Recall: 0.0723 - val_accuracy: 0.0353 - val_loss: 0.3220
Epoch 4/100


625/625 - 3s - 5ms/step - Precision: 0.5478 - Recall: 0.0791 - accuracy: 0.0485 - loss: 0.3250 - val_Precision: 0.5842 - val_Recall: 0.0769 - val_accuracy: 0.0481 - val_loss: 0.3209
Epoch 5/100


...

625/625 - 3s - 5ms/step - Precision: 0.5576 - Recall: 0.0855 - accuracy: 0.0519 - loss: 0.3217 - val_Precision: 0.5930 - val_Recall: 0.0785 - val_accuracy: 0.0579 - val_loss: 0.3199
Epoch 7/100


625/625 - 3s - 5ms/step - Precision: 0.5604 - Recall: 0.0878 - accuracy: 0.0516 - loss: 0.3204 - val_Precision: 0.5943 - val_Recall: 0.0798 - val_accuracy: 0.0546 - val_loss: 0.3196
Epoch 8/100


625/625 - 3s - 5ms/step - Precision: 0.5641 - Recall: 0.0897 - accuracy: 0.0509 - loss: 0.3196 - val_Precision: 0.5999 - val_Recall: 0.0784 - val_accuracy: 0.0557 - val_loss: 0.3193
Epoch 9/100
625/625 - 3s - 5ms/step - Precision: 0.5654 - Recall: 0.0913 - accuracy: 0.0520 - loss: 0.3186 - val_Precision: 0.5967 - val_Recall: 0.0816 - val_accuracy: 0.0469 - val_loss: 0.3194
Epoch 10/100
625/625 - 3s - 5ms/step - Precision: 0.5677 - Recall: 0.0931 - accuracy: 0.0527 - loss: 0.3178 - val_Precision: 0.5961 - val_Recall: 0.0847 - val_accuracy: 0.0512 - val_loss: 0.3194
Epoch 11/100
625/625 - 3s - 5ms/step - Precision: 0.5690 - Recall: 0.0947 - accuracy: 0.0519 - loss: 0.3172 - val_Precision: 0.6005 - val_Recall: 0.0828 - val_accuracy: 0.0489 - val_loss: 0.3193
Epoch 12/100
625/625 - 3s - 5ms/step - Precision: 0.5713 - Recall: 0.0954 - accuracy: 0.0515 - loss: 0.3166 - val_Precision: 0.6012 - val_Recall: 0.0820 - val_accuracy: 0.0509 - val_loss: 0.3194
Epoch 13/100
625/625 - 3s - 5ms/step - 

2025-07-17 01:52:52.747538: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 56 bytes spill stores, 56 bytes spill loads

2025-07-17 01:52:52.796323: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 4 bytes spill stores, 4 bytes spill loads

2025-07-17 01:52:52.799241: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 20 bytes spill stores, 20 bytes spill loads

2025-07-17 01:52:53.154311: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_23_0', 60 bytes spill stores, 60 bytes spill loads

2025-07-17 01:52:53.300549: I external/local_xla

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step
